In [2]:
from konlpy.tag import Mecab
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

2025-05-23 09:39:27.328899: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 09:39:27.411614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747960767.463227     529 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747960767.481776     529 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747960767.538711     529 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
test_data = pd.read_csv("./data/bank_app_reviews_test.csv")

In [4]:
import re
def clean_text(text):
    cleaned = re.sub(r'[^가-힣0-9a-zA-Z\s]', '', text)  # 한글, 숫자, 영문, 공백 제외한 모든 것 제거
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned.strip()


In [5]:
test_data['is_good'] = test_data['평점'].apply(lambda x: 1 if x >=4 else 0)
test_data['is_good']

0       1
1       1
2       0
3       0
4       1
       ..
9529    0
9530    1
9531    1
9532    1
9533    0
Name: is_good, Length: 9534, dtype: int64

In [6]:
test_data['사용자리뷰'] = test_data['사용자리뷰'].apply(clean_text)
test_data['사용자리뷰']

0                                              고경민계장님감사해요
1       저축목표피드 새로 생긴거 너무좋은데 분명 카테고리를 저축으로 했는데 왜 인식이 안되...
2         아니 이딴걸 편리하게 사용하는앱이라고 쳐만들엇나 이렇게 불편하게만든건 일부러그런거에요
3       몇 년째 만족하며 사용중이라 조금식 개선되어거는 모습에 만족하며 사용중입니다 하지만...
4                                스타뱅킹을 사용 하고나서부터 편안해서 좋아요
                              ...                        
9529    만보기 이벤트는 실망스러워요 후기 말투 다 똑같고 사기 맞죠 양심이 참 정직하게 확...
9530                   기능이 많아 다 사용해보진 못 했지만 대체적으로 편한거 같아요
9531                                                편리하네요
9532                                            사용하기 편리해요
9533    너무 후져서 오랜만에 어플이용하다가 욕했답니다 인증방식이 2010년대에 머물러계심 ...
Name: 사용자리뷰, Length: 9534, dtype: object

In [7]:
test_data['is_good'] = test_data['평점'].apply(lambda x: 1 if x >=4 else 0)
test_data['is_good']

0       1
1       1
2       0
3       0
4       1
       ..
9529    0
9530    1
9531    1
9532    1
9533    0
Name: is_good, Length: 9534, dtype: int64

In [8]:
mecab = Mecab()

In [9]:
dokenized_docs =  test_data['사용자리뷰'].apply(mecab.morphs)

In [10]:
dokenized_docs[0]

['고경민', '계장', '님', '감사', '해요']

# train에서 사용했던 tokenizer를 불러와서 one hot encoding

In [12]:
import joblib

In [15]:
token = joblib.load("./model/bank_app_tokeizer.joblib")
max_len = joblib.load("./model/bank_app_max_length.joblib")
print(max_len)

302


In [16]:
x = token.texts_to_sequences(dokenized_docs)
print(x[0])

[6248, 327, 111, 71]


# train에서 사용했던 패딩길이 (모델 넣을 컬럼 수)

In [17]:
max_len = joblib.load("./model/bank_app_max_length.joblib")
max_len

302